In [1]:
import logging
import os
import argparse
import math
import numpy as np

import traceback

from tqdm import tqdm

import pandas as pd

from src.config.config import Config, load_config_from_json
from src.evaluate.rouge_evaluator import ScoreSummary
from src.loader.class_loader import Dataset, SOURCE, Cluster
from src.utils.factory import create_model, create_evaluator
from src.model.sds.combination import CombinationRanker
from src.loader.class_loader import load_cluster

In [2]:
config = load_config_from_json()

try:
    train_set = load_cluster(
        config.train_path,
    )
    logging.warning("[PIPELINE] - Load train set from {}. Done.".format(config.train_path))
except Exception as e:
    train_set = None
    logging.warning("[PIPELINE] - Load train set from {}. Failed. Using None.".format(config.train_path))
    print(e)

try:
    valid_set = load_cluster(
        config.valid_path,
    )
    logging.warning("[PIPELINE] - Load valid set from {}. Done.".format(config.valid_path))
except Exception as e:
    valid_set = None
    logging.warning("[PIPELINE] - Load valid set from {}. Failed. Using None.".format(config.valid_path))
    print(e)

try:
    test_set = load_cluster(
        "/home/hvn/Documents/dskt/vlsp-final-year/dataset/vlsp_abmusu_test_data.jsonl",
    )
    logging.warning("[PIPELINE] - Load test set from {}. Done.".format("/home/hvn/Documents/dskt/vlsp-final-year/dataset/vlsp_abmusu_test_data.jsonl"))
except Exception as e:
    test_set = None
    logging.warning("[PIPELINE] - Load test set from {}. Failed. Using None.".format("/home/hvn/Documents/dskt/vlsp-final-year/dataset/vlsp_abmusu_test_data.jsonl"))
    print(e)

Total number of cluster:  200


200it [00:33,  5.89it/s]


Total number of cluster:  100


100it [00:16,  6.06it/s]


Total number of cluster:  300


300it [00:43,  6.82it/s]


In [3]:
train_set.set_source(SOURCE.SENT_SPLITTED_TOKEN.value)
valid_set.set_source(SOURCE.SENT_SPLITTED_TOKEN.value)
test_set.set_source(SOURCE.SENT_SPLITTED_TOKEN.value)

train_scores = []
valid_scores = []
test_scores = []

In [4]:
from src.model.sds.combination import CombinationRanker

config = load_config_from_json()
model_config = config.models[15]

model = CombinationRanker(model_config)
model.training(valid_set)

Loading codes from /home/hvn/Documents/dskt/vlsp-final-year/external/sentence_transformer/vn_sbert_deploy/bpe/bpe.codes ...
Read 64000 codes from the codes file.


In [5]:
weight = {
    "tfidf": 0.1,
    "lexrank": 0.1,
    "textrank": 0.1
}
    
print("get local score on train set")
for cluster in tqdm(train_set.clusters):
    document_score = []

    for doc in cluster.documents:
        document_score.append(model.get_score(doc.get_all_sents(), 1))

    train_scores.append(document_score)

print("get local score on valid set")
for cluster in tqdm(valid_set.clusters):
    document_score = []

    for doc in cluster.documents:
        document_score.append(model.get_score(doc.get_all_sents(), 1))

    valid_scores.append(document_score)
    
print("get local score on test set")
for cluster in tqdm(test_set.clusters):
    document_score = []

    for doc in cluster.documents:
        document_score.append(model.get_score(doc.get_all_sents(), 1))

    test_scores.append(document_score)

get local score on train set


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:33<00:00,  5.96it/s]


get local score on valid set


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:15<00:00,  6.35it/s]


get local score on test set


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:44<00:00,  6.76it/s]


In [6]:
from src.evaluate.rouge_evaluator import RougeScore

class RougeScoreStorage:
    def __init__(self):
        self.df = pd.DataFrame(columns=[
            'cluster_id',
            'rouge_1_p',
            'rouge_1_r',
            'rouge_1_f',
            'rouge_2_p',
            'rouge_2_r',
            'rouge_2_f',
            'rouge_l_p',
            'rouge_l_r',
            'rouge_l_f',
        ])
    
    def add_score(self, cluster_id: int, score: RougeScore):
        self.df = self.df.append({
            'cluster_id': cluster_id,
            'rouge_1_p': score.rouge1.p,
            'rouge_1_r': score.rouge1.r,
            'rouge_1_f': score.rouge1.f1,
            'rouge_2_p': score.rouge2.p,
            'rouge_2_r': score.rouge2.r,
            'rouge_2_f': score.rouge2.f1,
            'rouge_l_p': score.rougeL.p,
            'rouge_l_r': score.rougeL.r,
            'rouge_l_f': score.rougeL.f1,
        }, ignore_index=True)
        
    def summary_score(self):
        summary_df = pd.DataFrame(columns=[
            'name',
            'mean',
            'min',
            'max',
            'std',
        ])

        metric_cols = [
            'rouge_1_p',
            'rouge_1_r',
            'rouge_1_f',
            'rouge_2_p',
            'rouge_2_r',
            'rouge_2_f',
            'rouge_l_p',
            'rouge_l_r',
            'rouge_l_f', ]

        for col in metric_cols:
            describe = self.df[col].describe()
            summary_df = summary_df.append({
                'name': col,
                'mean': describe['mean'],
                'min': describe['min'],
                'max': describe['max'],
                'std': describe['std'],
            }, ignore_index=True)
        
        return summary_df

In [7]:
from src.evaluate.rouge_evaluator import ScoreSummary
from src.utils.factory import create_model, create_evaluator
from src.model.mmr_query import MMRSummarizerQuery

evaluator = create_evaluator(config.eval_config)

In [8]:
n_sent = [11, 0.2]

def get_rouge_score(weights, sigma):
    local_config = config.models[15]
    local_config.sigma = sigma
    
    mmr = MMRSummarizerQuery(local_config) 
    
    train_storage = RougeScoreStorage()
    valid_storage = RougeScoreStorage()
    
    print("Test on training set")
    
    """train_set.set_source(SOURCE.SENT_SPLITTED_TEXT.value)
    for idx, cluster in tqdm(enumerate(train_set.clusters)):
        chosen_sent = [] 

        sent_count = len(cluster.get_all_sents())
        for SENT_COUNT in n_sent:
            if 0 <= SENT_COUNT < 1:
                sent_count = min(int(math.ceil(len(cluster.get_all_sents()) * SENT_COUNT)), sent_count)
            else:
                sent_count = min(int(SENT_COUNT), sent_count)


        for idxc, doc in enumerate(cluster.documents):
            doc.set_source(SOURCE.SENT_SPLITTED_TEXT.value)
            sents = doc.get_all_sents()

            scores = train_scores[idx][idxc]

            combine_score = np.zeros((len(scores["tfidf"])), dtype=float)

            for key in weights.keys():
                combine_score += scores[key] * weights[key]

            if (sent_count >= len(combine_score)):
                chosen_idx = list(range(len(combine_score)))
            else:
                chosen_idx = np.argpartition(combine_score, -sent_count)[-sent_count:]

            for i in chosen_idx:
                chosen_sent.append(sents[i]) 

        pred_sent, _ = mmr(chosen_sent, sent_count, cluster.get_all_anchor())

        train_storage.add_score(
            cluster.cluster_idx,
            evaluator(
                '.'.join(pred_sent),
                '.'.join(cluster.get_summary()),
            )
        )"""
    
    print("Test on valid set")
    
    valid_set.set_source(SOURCE.SENT_SPLITTED_TEXT.value)
    for idx, cluster in tqdm(enumerate(valid_set.clusters)):
        chosen_sent = [] 
        
        sent_count = len(cluster.get_all_sents())
        for SENT_COUNT in n_sent:
            if 0 <= SENT_COUNT < 1:
                sent_count = min(int(math.ceil(len(cluster.get_all_sents()) * SENT_COUNT)), sent_count)
            else:
                sent_count = min(int(SENT_COUNT), sent_count)
        
        
        for idxc, doc in enumerate(cluster.documents):
            doc.set_source(SOURCE.SENT_SPLITTED_TOKEN.value)
            sents = doc.get_all_sents()
            
            scores = valid_scores[idx][idxc]
            
            combine_score = np.zeros((len(scores["tfidf"])), dtype=float)
        
            for key in weights.keys():
                combine_score += scores[key] * weights[key]
            
            if (sent_count >= len(combine_score)):
                chosen_idx = list(range(len(combine_score)))
            else:
                chosen_idx = np.argpartition(combine_score, -sent_count)[-sent_count:]
            
            for i in chosen_idx:
                chosen_sent.append(sents[i]) 
                
        pred_sent, _ = mmr(chosen_sent, sent_count, cluster.get_all_anchor())
        
        cluster.set_source(SOURCE.SENT_SPLITTED_TOKEN.value)
        valid_storage.add_score(
            cluster.cluster_idx,
            evaluator(
                '.'.join(pred_sent),
                '.'.join(cluster.get_summary()),
            )
        )
        
    print("Using weight\n", weights)
    print("Sigma: ", sigma)
    # print("Train result\n", train_storage.summary_score())
    print("Valid result\n", valid_storage.summary_score())

In [17]:
def get_rouge_score_and_saved(weights, sigma, filename):
    local_config = config.models[15]
    local_config.sigma = sigma
    
    mmr = MMRSummarizerQuery(local_config) 
    
    train_storage = RougeScoreStorage()
    valid_storage = RougeScoreStorage()
    
    print("Test on training set")
    
    """train_set.set_source(SOURCE.SENT_SPLITTED_TEXT.value)
    for idx, cluster in tqdm(enumerate(train_set.clusters)):
        chosen_sent = [] 

        sent_count = len(cluster.get_all_sents())
        for SENT_COUNT in n_sent:
            if 0 <= SENT_COUNT < 1:
                sent_count = min(int(math.ceil(len(cluster.get_all_sents()) * SENT_COUNT)), sent_count)
            else:
                sent_count = min(int(SENT_COUNT), sent_count)


        for idxc, doc in enumerate(cluster.documents):
            doc.set_source(SOURCE.SENT_SPLITTED_TEXT.value)
            sents = doc.get_all_sents()

            scores = train_scores[idx][idxc]

            combine_score = np.zeros((len(scores["tfidf"])), dtype=float)

            for key in weights.keys():
                combine_score += scores[key] * weights[key]

            if (sent_count >= len(combine_score)):
                chosen_idx = list(range(len(combine_score)))
            else:
                chosen_idx = np.argpartition(combine_score, -sent_count)[-sent_count:]

            for i in chosen_idx:
                chosen_sent.append(sents[i]) 

        pred_sent, _ = mmr(chosen_sent, sent_count, cluster.get_all_anchor())

        train_storage.add_score(
            cluster.cluster_idx,
            evaluator(
                '.'.join(pred_sent),
                '.'.join(cluster.get_summary()),
            )
        )"""
    
    print("Test on valid set")
    
    valid_set.set_source(SOURCE.SENT_SPLITTED_TEXT.value)
    for idx, cluster in tqdm(enumerate(valid_set.clusters)):
        chosen_sent = [] 
        
        sent_count = len(cluster.get_all_sents())
        for SENT_COUNT in n_sent:
            if 0 <= SENT_COUNT < 1:
                sent_count = min(int(math.ceil(len(cluster.get_all_sents()) * SENT_COUNT)), sent_count)
            else:
                sent_count = min(int(SENT_COUNT), sent_count)
        
        
        for idxc, doc in enumerate(cluster.documents):
            doc.set_source(SOURCE.SENT_SPLITTED_TOKEN.value)
            sents = doc.get_all_sents()
            
            scores = valid_scores[idx][idxc]
            
            combine_score = np.zeros((len(scores["tfidf"])), dtype=float)
        
            for key in weights.keys():
                combine_score += scores[key] * weights[key]
            
            if (sent_count >= len(combine_score)):
                chosen_idx = list(range(len(combine_score)))
            else:
                chosen_idx = np.argpartition(combine_score, -sent_count)[-sent_count:]
            
            for i in chosen_idx:
                chosen_sent.append(sents[i]) 
                
        pred_sent, _ = mmr(chosen_sent, sent_count, cluster.get_all_anchor())
        
        cluster.set_source(SOURCE.SENT_SPLITTED_TOKEN.value)
        valid_storage.add_score(
            cluster.cluster_idx,
            evaluator(
                '.'.join(pred_sent),
                '.'.join(cluster.get_summary()),
            )
        )
     
    print("Predict on test set")
    
    test_set.set_source(SOURCE.SENT_SPLITTED_TEXT.value)
    predictions = []
    
    for idx, cluster in tqdm(enumerate(test_set.clusters)):
        chosen_sent = [] 
        
        sent_count = len(cluster.get_all_sents())
        for SENT_COUNT in n_sent:
            if 0 <= SENT_COUNT < 1:
                sent_count = min(int(math.ceil(len(cluster.get_all_sents()) * SENT_COUNT)), sent_count)
            else:
                sent_count = min(int(SENT_COUNT), sent_count)
        
        
        for idxc, doc in enumerate(cluster.documents):
            doc.set_source(SOURCE.SENT_SPLITTED_TEXT.value)
            sents = doc.get_all_sents()
            
            scores = test_scores[idx][idxc]
            
            combine_score = np.zeros((len(scores["tfidf"])), dtype=float)
        
            for key in weights.keys():
                combine_score += scores[key] * weights[key]
            
            if (sent_count >= len(combine_score)):
                chosen_idx = list(range(len(combine_score)))
            else:
                chosen_idx = np.argpartition(combine_score, -sent_count)[-sent_count:]
            
            for i in chosen_idx:
                chosen_sent.append(sents[i]) 
                   
        pred_sent, _ = mmr(chosen_sent, sent_count, cluster.get_all_anchor())
        predictions.append(' '.join(pred_sent))
        
    print("Start write to txt")
    with open(os.path.join("/home/hvn/Documents/dskt/vlsp-final-year/data/result/outer/combination", "results" + filename + ".txt"), "w") as f:
        for summary in predictions:
            summary.replace('_', ' ')
            f.write(summary)
            f.write('\n')
    print("Done write to txt")
                
    print("Using weight\n", weights)
    print("Sigma: ", sigma)
    #print("Train result\n", train_storage.summary_score())
    print("Valid result\n", valid_storage.summary_score())

In [10]:
for sigma in [0.0, 0.2, 0.8, 1.0]:
    for i in [0.0, 0.3, 0.7, 1.0]:
        for j in [0.0, 0.3, 0.7, 1.0]:
            if i + j > 1.0:
                continue 
            k = 1.0 - i - j 

            weight = {
                "tfidf": i,
                "lexrank": j,
                "textrank": k
            }

            get_rouge_score(weight, sigma)

Loading codes from /home/hvn/Documents/dskt/vlsp-final-year/external/sentence_transformer/vn_sbert_deploy/bpe/bpe.codes ...
Read 64000 codes from the codes file.


Test on training set
Test on valid set


100it [00:36,  2.76it/s]


Using weight
 {'tfidf': 0.0, 'lexrank': 0.0, 'textrank': 1.0}
Sigma:  0.0
Valid result
         name      mean       min       max       std
0  rouge_1_p  0.360405  0.107784  0.824324  0.133059
1  rouge_1_r  0.521403  0.223404  0.865854  0.131028
2  rouge_1_f  0.413784  0.156716  0.734940  0.111168
3  rouge_2_p  0.178915  0.004367  0.736264  0.135124
4  rouge_2_r  0.267028  0.008475  0.761468  0.162344
5  rouge_2_f  0.205260  0.005764  0.653659  0.133654
6  rouge_l_p  0.334677  0.101796  0.824324  0.133831
7  rouge_l_r  0.483207  0.191489  0.865854  0.135778
8  rouge_l_f  0.383806  0.134328  0.734940  0.115521


Loading codes from /home/hvn/Documents/dskt/vlsp-final-year/external/sentence_transformer/vn_sbert_deploy/bpe/bpe.codes ...
Read 64000 codes from the codes file.


...

Using weight
 {'tfidf': 0.7, 'lexrank': 0.3, 'textrank': 5.551115123125783e-17}
Sigma:  1.0
Valid result
         name      mean       min       max       std
0  rouge_1_p  0.414543  0.214876  0.759259  0.134937
1  rouge_1_r  0.464090  0.196581  0.830769  0.137893
2  rouge_1_f  0.423038  0.229630  0.694915  0.106567
3  rouge_2_p  0.200594  0.000000  0.593023  0.141048
4  rouge_2_r  0.241564  0.000000  0.695122  0.152085
5  rouge_2_f  0.208225  0.000000  0.520270  0.127008
6  rouge_l_p  0.383029  0.183099  0.759259  0.133392
7  rouge_l_r  0.430300  0.127273  0.815385  0.139881
8  rouge_l_f  0.391494  0.172840  0.669492  0.109625


Loading codes from /home/hvn/Documents/dskt/vlsp-final-year/external/sentence_transformer/vn_sbert_deploy/bpe/bpe.codes ...
Read 64000 codes from the codes file.


Test on training set
Test on valid set


100it [00:28,  3.51it/s]

Using weight
 {'tfidf': 1.0, 'lexrank': 0.0, 'textrank': 0.0}
Sigma:  1.0
Valid result
         name      mean       min       max       std
0  rouge_1_p  0.412261  0.200000  0.759259  0.134925
1  rouge_1_r  0.453709  0.196581  0.938462  0.141551
2  rouge_1_f  0.417645  0.235714  0.694915  0.108121
3  rouge_2_p  0.197784  0.000000  0.592105  0.146677
4  rouge_2_r  0.230236  0.000000  0.817073  0.160724
5  rouge_2_f  0.202226  0.000000  0.520270  0.134149
6  rouge_l_p  0.382546  0.175758  0.759259  0.133485
7  rouge_l_r  0.422054  0.127273  0.907692  0.141824
8  rouge_l_f  0.388056  0.172840  0.669492  0.110879


In [19]:
weight = {
    "tfidf": 0.0,
    "lexrank": 0.85,
    "textrank": 0.15,
}

get_rouge_score_and_saved(weight, 0.7, "_2")

Loading codes from /home/hvn/Documents/dskt/vlsp-final-year/external/sentence_transformer/vn_sbert_deploy/bpe/bpe.codes ...
Read 64000 codes from the codes file.


Test on training set
Test on valid set


100it [00:37,  2.69it/s]


Predict on test set


300it [01:19,  3.78it/s]


Start write to txt
Done write to txt
Using weight
 {'tfidf': 0.0, 'lexrank': 0.75, 'textrank': 0.25}
Sigma:  0.7
Valid result
         name      mean       min       max       std
0  rouge_1_p  0.412075  0.187500  0.983607  0.147609
1  rouge_1_r  0.611455  0.200000  0.986667  0.134345
2  rouge_1_f  0.480747  0.244186  0.865497  0.129110
3  rouge_2_p  0.237838  0.020349  0.925373  0.163577
4  rouge_2_r  0.383992  0.041667  0.947368  0.176823
5  rouge_2_f  0.284496  0.028398  0.779874  0.163313
6  rouge_l_p  0.388049  0.170404  0.983607  0.153828
7  rouge_l_r  0.574716  0.127273  0.986667  0.144526
8  rouge_l_f  0.452599  0.172840  0.865497  0.139614
